In [1]:
import sys
import os
from os.path import join
from pathlib import Path
sys.path.append(join(Path(os.getcwd()).parent, 'lib'))

In [2]:
! python3 -m pip install unidecode

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
%matplotlib inline

import json
import os
import pandas as pd
import re
import numpy as np
import pickle
import collections
from importlib import reload
from  mainlib import fasorial, lista, sdict, mm, mm_sep
from unidecode import unidecode
from of_proc import *
from os.path import join
from os import listdir as ld
from unidecode import unidecode
from matplotlib import pyplot as plt

Using TensorFlow backend.


In [4]:
def read_info(path):
    global PATH, OUTPUT_PATH, PROCESSED_PATH, PERFIS_PATH, HORARIOS_PATH, PATH_MM, PATH_FAS, \
    DEG, MOR, PER, FAS, OUTPUT_HEADER,FAS_CONCAT, MM_CONCAT, PICKLES_CONCAT, OUT_INDEX, COLS_LISTA, \
    OUT, DATA_DIR, LISTA_CAD_PATH, PROCESSADOS_PICKLE, ERROS_PICKLE, LOGS_DIR
    with open(path) as f:
        info = json.load(f)
        PATH_FAS = info['paths']['fasoriais2']['input']
        PERFIS_PATH = info['paths']['perfis']['perfis']
        HORARIOS_PATH = info['paths']['perfis']['horarios']
        PATH_MM = info['paths']['mm']['input']
        DEG = info['paths']['nn_models']['degrau']
        MOR = info['paths']['nn_models']['morro']
        PER = info['paths']['nn_models']['perfil']
        FAS = info['paths']['nn_models']['fasorial']
        COLS_LISTA = info['lista2']['columns']  
        OUTPUT_HEADER = info['output']['header']
        OUT_INDEX = info['output']['header']
        OUT = info['paths']['output']
        DATA_DIR = info['paths']['base_dir']
        LISTA_CAD_PATH = info['paths']['lista']
        PROCESSADOS_PICKLE = info['paths']['processados']
        ERROS_PICKLE = info['paths']['erros']
        LOGS_DIR = info['paths']['logs']


# Parâmetros

In [5]:
json_file = '../../info/info-EDP-ES.json'
gd = True

# Informações

In [6]:
def dk(d,k=0): return list(d.values())[k]

In [7]:
read_info(json_file)

In [8]:
def get_horario_info2(json_file):
    with open(json_file) as jinfo:
        info = json.load(jinfo)
        HORARIOS_PATH = info['paths']['perfis']['horarios']
        PERFIS_PATH = info['paths']['perfis']['perfis']
        ATIV_PATH = info['paths']['perfis']['mapa']


    perfis = pd.read_csv(PERFIS_PATH, sep = ';')
    ativs = pd.read_csv(ATIV_PATH, sep = ';')

    #perfis.columns = ['id', 'instalacao', 'cpf_cnpj', 'razao social', 'endereco_cadastro', 'ramo_de_atividade','None1', 'None2']
    
    dates = re.compile("[0-9]{2}:[0-9]{2}")
    dateparser = lambda x: pd.datetime.strptime(x, '%H:%M') if type(x) == str  and dates.match(x) else pd.NaT
    cols = ['name','inicio_dia_util','fim_dia_util', 'inicio_sabado', 'fim_sabado', 'inicio_domingo', 'fim_domingo', 'inicio_feriado', 'fim_feriado']
    horarios = pd.read_csv(HORARIOS_PATH, sep = ';',skipinitialspace=True, skiprows = 2, na_values = np.NaN, names = cols,
                          date_parser = dateparser, parse_dates = [1,2,3,4,5,6,7,8])
    
    
    horarios.set_index('name', inplace = True)
    horarios.dropna(inplace=True)

    perfis.set_index('instalacao', inplace=True)
    perfis = (perfis.groupby(level=0).first())

    
    return horarios, perfis, ativs

def get_files_4(nomes, dim,  CAMINHO):
    not_found = []
    dif = {}
    for i in nomes:
        if i in dim:
            tmp_list = []
            search = ab.search_list_2(str(i),os.listdir(CAMINHO))
            if search:
                dif[i] = search
            else:
                not_found.append(i)                
    return dif, not_found

#Criar dicionario de categorias
def insta2horario(HORARIOS_PATH):
    horarios = pd.read_csv(HORARIOS_PATH, sep=',')
    horarios.drop(columns=horarios.columns[0],inplace=True)
    horarios.columns = ['nome', 'cat', 'dias_uteis_i',
           'dias_uteis_f', 'sabado_i', 'sabado_f', 'domingo_i',
           'domingo_f', 'feriado_i', 'feriado_f']
    horarios['nome'] = horarios['nome'].apply(unidecode)
    dfcat = horarios.iloc[:,:2]
    dfcat.set_index('nome',inplace=True)
    scat = dfcat['cat']
    dcat = scat.to_dict()
    horarios.set_index('nome', inplace=True)
    func = {k: [pd.Timestamp(v) for v in horarios.loc[k,'dias_uteis_i':'feriado_f']] for k in horarios.index}
    return func

def per2cluster(func, ATIV):
    dfativ = pd.read_csv(ATIV, sep=';', names = ['ramo', 'perfil'])
    dfativ=dfativ.applymap(unidecode)
    dfativ.set_index('ramo', inplace=True)
    a2p = dfativ.loc[:,'perfil'].T.to_dict()
    a2p.pop('Ramo de Atividade')
    p2a = dict((v,k) for k,v in a2p.items())
    perfunc=PERFIS['ramo_de_atividade'].map(a2p).map(func)
    return perfunc

In [9]:
global df_mm_new
mmm = mm(json_file)
df_mm_new = pd.read_csv(PATH_MM,date_parser = mmm.dateparser, index_col=[0], sep=';')
dummy_mm = ""

## l_______________________________________________________________________

### Carregar modelos neurais

# Coleções principais

In [10]:
global LISTA_CAD
global PERFIL, DEGRAU, MORRO, NN_FAS
global files_fas, files_mm
global HORARIOS, PERFIS, ATIVS, DPERFIS, DPERFIS_NORMAL, DPERFIS_TROCA
global dim, dmi, df_fas, dfiles_mm, dfiles_fas_troca, dfiles_mm_troca, dfiles_fas_normal, dfiles_mm_normal
global o_i_normal, o_i_troca, o_instas, list_i_normal, list_i_troca, list_intas, gd

In [11]:
PERFIL, DEGRAU, MORRO, NN_FAS = load_models(PER, DEG, MOR, FAS)

Instructions for updating:
Colocations handled automatically by placer.


## l_______________________________________________________________________

In [12]:
LISTAF = pd.read_csv(LISTA_CAD_PATH, names = COLS_LISTA, header=0, sep=';',engine='python')


In [13]:
LISTAF['instalacao'] = pd.DataFrame(LISTAF['instalacao'].astype(int).apply(lambda x: '{0:0>10}'.format(x)))
LISTAF['medidor'] = pd.DataFrame(LISTAF['medidor'].astype(int).apply(lambda x: '{0:0>8}'.format(x)))


In [14]:
faltantes = [
'0000081002',
'0000357754',
'0000586609',
'0001659074',
'0009502437',
'0160214293',
'0160557791',
'0160581342',
'0160669410'   
]

In [15]:
LISTAF = LISTAF[LISTAF['instalacao'].isin(faltantes)]

In [16]:
LISTA_CAD = get_list(json_file)
LISTA_CAD.df =  LISTAF
HORARIOS, PERFIS, ATIVS = get_horario_info2(json_file)
dim, dmi = dicts(LISTA_CAD)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc7 in position 15: invalid continuation byte

#### Estruturas auxiliares

In [17]:
o_i_normal = collections.OrderedDict(sorted(LISTA_CAD.mes_days(troca=False).items()))
o_i_troca = collections.OrderedDict(sorted(LISTA_CAD.mes_days(troca=True).items()))
o_instas =  collections.OrderedDict(sorted({**o_i_normal, **o_i_troca}.items()))

NameError: name 'LISTA_CAD' is not defined

#### Arquivos

In [ ]:
import A_base
reload(A_base)

In [ ]:
import A_base as ab

In [ ]:
dfiles_fas, nf_fas= get_files_4(list(o_instas.keys()), dim,  PATH_FAS)
ddirs_fas = {k:join(PATH_FAS,v) for k,v in dfiles_fas.items()}

#### Lista , perfis e horários

In [ ]:
# Dicionário de instalacao para horario de funcionamento
func =  insta2horario(HORARIOS_PATH)

In [ ]:
# Dicionário de perfis para clusters
perfunc = per2cluster(func)

In [ ]:
percats = PERFIS['cat'].fillna(value=0).apply(int).to_dict()

 l___________________________________________________________________________________________________________________________________________

#### Pickles

## l_______________________________________________________________________

# Importante

In [ ]:
log_count=0

In [ ]:
sem_hora = []
err_msg = {}
missing = []

In [ ]:
with open(PROCESSADOS_PICKLE,'rb') as p:
    processados=pickle.load(p)
with open(ERROS_PICKLE,'rb') as p:
    erros = pickle.load(p)

In [ ]:
log_file = open(os.path.join(LOGS_DIR,str(log_count)), 'at+')

In [ ]:
def log(string):
    print(string)
    log_file.write(string)

In [ ]:
def dici2list(d_insta_to_days):
    ddays={}
    ddays[4],ddays[5],ddays[6],ddays[7] = d_insta_to_days[12], d_insta_to_days[1], d_insta_to_days[2], d_insta_to_days[3], d_insta_to_days[4]
    return ddays
                

In [ ]:
import traceback

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn' . option None

In [ ]:
padrao = [pd.Timestamp('2019-09-20 00:00:00'),
 pd.Timestamp('2019-09-20 23:59:00'),
 pd.Timestamp('2019-09-20 00:00:00'),
 pd.Timestamp('2019-09-20 23:59:00'),
 pd.Timestamp('2019-09-20 00:00:00'),
 pd.Timestamp('2019-09-20 23:59:00'),
 pd.Timestamp('2019-09-20 00:00:00'),
 pd.Timestamp('2019-09-20 23:59:00')]

In [ ]:
def loopy2(d_insta_to_days, d_insta_to_files_fas, d_insta_to_files_mm,PATH_FAS, PATH_MM, output_dir,sepfas, sepmm, gd):
    df_out = pd.DataFrame(index=OUT_INDEX)
    global difas, dimm
    global dfas, dmm,  dimm
    difas, dimm = sdict(), sdict()
    c = 0
    c1 = 0
    for instalacao in d_insta_to_days:
        c1 = c1 + 1
        if (instalacao  not in processados):
            log(str(c)+'--'+'######## ' + instalacao + ' #########')
            medidor = dim[instalacao][0] if type(dim[instalacao]) is list else dim[instalacao]
            try:      
                print('------ 0')
                #ramo = percats[instalacao] if instalacao in percats else 0
                ramo = 0
                print('------ 1')
                #hours = perfunc[instalacao] if instalacao in perfunc else padrao
                hours = padrao
                print('------ 2')
                ddays = d_insta_to_days[instalacao] if instalacao in d_insta_to_days else default_ddays_2()
                print('------ 3')
                dfas = fasorial(json_file, name = instalacao, ddays=ddays)     
                print('------ 4')
                dfas.outputs['perfil'] = ramo
                print('------ 5')
                dfas.read_fasorial(d_insta_to_files_fas[instalacao], dtype=None, index_col=[5], sep = sepfas)
                print('------ 6')
                dmm = mm(json_file, name = instalacao, ddays = ddays, ramo=ramo)
                print('------ 7')
                if dim[instalacao] in df_mm_new.columns:
                    dmm.df = pd.DataFrame(df_mm_new[dim[instalacao]])
                else:
                    dmm.df = pd.DataFrame(df_mm_new[str(int(dim[instalacao]))])                 

                print('------ 8')
                proc_fas(dfas,ddays, NN_FAS, json_file, hours = dfas.hours, gd=gd)
                processa_mm(dmm, ddays, PERFIL, DEGRAU, MORRO, json_file)
                print(d_insta_to_files_fas[instalacao])
                print('################################################')
                #Cria um dicionário de outputs por mês para o fasorial
                for y in dfas.months:
                    dfas.outputs[y] = {}
                    for month  in dfas.months[y]:
                        dfas.outputs[y][month] = dfas.months[y][month].outputs                
                df_out = cat_out2(df_out, dmm.outputs, dfas.outputs, ddays,OUT_INDEX)

                difas[instalacao] = dfas
                dimm[instalacao] = dmm                
                c = c+1                
                #if c1 % 100 ==0 :  
                if False:
                    #return df_out, dfas, dmm
                    log('Salvando csv!')
                    df_out.T.to_csv(output_dir + 'e1_' +str(c1-100) + '_to_' + str(c1) +'.csv')                    
                    pickle.dump(processados, open(PROCESSADOS_PICKLE,'wb'))
                    pickle.dump(erros, open(ERROS_PICKLE,'wb'))                    
                    df_out=pd.DataFrame()
                processados.append(instalacao)
            except Exception as e:
                erros.append(instalacao)
                err_msg[instalacao] = traceback.format_exc()
                print('++++++Erro 2++++++++++ \n'+ str(traceback.format_exc())+'\n++++++++++++++++++++')
    df_out.T.to_csv(output_dir + 'new.csv')        

## l_______________________________________________________________________

In [ ]:
from mainlib import mm

In [ ]:
gd

In [ ]:
processados, erros = [], []
sepmm=','
sepfas=';'
m=loopy2(o_instas, ddirs_fas, dummy_mm, PATH_FAS, PATH_MM, OUT, sepfas, sepmm, gd)

## l_______________________________________________________________________

##### Substituir coluna "Perfil cadastrado" por texto

In [ ]:
l=LISTA_CAD.df.drop(index=0)

In [ ]:
ra=PERFIS['ramo_de_atividade']

In [ ]:
dra=ra.to_dict()

In [ ]:
o = pd.read_csv('../../outputs/all/EDP-ES/new.csv')

In [ ]:
o['Perfil cadastrado']=o['Nome'].map(dra)

In [ ]:
o.to_csv('../../outputs/all/EDP-ES/new2.csv')

## l_______________________________________________________________________

# 

## l_______________________________________________________________________

## l_______________________________________________________________________

#### Leituras faltando - Fasorial

In [ ]:
import mainlib
reload(mainlib)
from mainlib import dframe

dfr = dframe()
data = dfr.data
dp = dfr.dateparser

ddirs_fas = pload('../pickles/ddirs_fas.pkl')

emp = {}
for n,(k,v) in enumerate(ddirs_fas.items()):
    df = pd.read_csv(v,sep=';',date_parser = dp,parse_dates=[5],names=data['fasorial']['columns'])
    emp[k]=df.set_index('data').iloc[:,0].resample('15min').first().isna().sum()
    print(n,k,emp[k])

demp=pd.DataFrame(index=emp.keys(), data=emp.values(),columns = ['Leituras faltantes'])

In [ ]:
demp.to_csv('../outputs/fas_emp.csv')

## l_______________________________________________________________________

#### Leituras faltando - Fasorial

In [ ]:
PATH_MM

In [ ]:
mm = pd.read_csv(PATH_MM, parse_dates = [0],date_parser=dp, index_col=[0])

In [ ]:
mm[mm.index.isnull()]

In [ ]:
mm.isnull().sum().sum()

In [ ]:
mm.describe()